In [7]:
import mpg.games.mpg as mpg
import mpg.visualisation.game as vg
import mpg.graph.random_graph as rg
import mpg.wrapper as mpgcpp
import time
import numpy as np

G=rg.gnp_random_mpg(n=260,p=0.001,seed=46,distribution="uniform",low=-5,high=5)
M=vg.MPGVisualisation(G)
start=time.time()
S1,S2=mpgcpp.optimal_strategy_pair(G,edgetype=np.float32)
end=time.time()
M.set_node_color_mapping(vg.WinnerVisualiser(G,S1,S2))
M.set_edge_color_mapping(vg.StrategyVisualiser(S1,S2))
M

MPGVisualisation(layout=Layout(height='500px', width='100%'))

In [29]:
game=G
mapper = dict(zip(game.nodes, range(len(game.nodes))))
game_cxx = []
n=len(game.nodes)
for u, v in game.edges:
    game_cxx.append((mapper[u], mapper[v], game.edges[u, v]['weight']))

In [3]:
type(game_cxx[0][0])

int

In [8]:
print(end-start)

0.09681892395019531


In [3]:
import mpg.wrapper.mpgcpp as mpgc

In [ ]:
mpgc.